# Prepare Case study - INDiECAR-RTN

---

## Prepare case study

Below, you find a step-by-step instruction on how to run a case study.

Define functions:

In [1]:
from IDRIC_toolkit import prepare_CASE
import csv
import os
import pandas as pd
import sys

def identify_all_csv (file_path):
    all_files = os.listdir(file_path)    
    csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
    return csv_files

def get_delimiter(file_path: str) -> str:
    with open(file_path, 'r') as csvfile:
        delimiter = str(csv.Sniffer().sniff(csvfile.read()).delimiter)
        return delimiter

def make_proper_csv (file_path,seperator):
    delimiter = get_delimiter(file_path)
    df = pd.read_csv(file_path, delimiter=delimiter)
    df.to_csv(file_path,index= False, sep=seperator)

def change_delimiter_for_all_csv_in_path (file_path,separator):
    csv_files = identify_all_csv(file_path)
    for i in csv_files:
        #print(f'{file_path}{i}')
        make_proper_csv(f'{file_path}{i}',separator)

def list_subfolders(general_path):
    return [name for name in os.listdir(general_path) if os.path.isdir(os.path.join(general_path, name)) ]


c:\Users\bst3191\anaconda3\envs\geospatial_environment\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### STEP 1 of 2: CREATE cases study

Select the folder where you keep all case studies.
(On the dropbox project folder, this relates to the folder *WP3 cluster case studies*, but this can differ for your local machine)

In [2]:
Path = f'../No_collaboration/'
Casenames = list_subfolders(Path)
Casenames

['SP_PT_AG Cementos Balboa SA',
 'SP_PT_Cementos Portland Valderrivas SA',
 'SP_PT_CEMEX SAB de CV',
 'SP_PT_Corporacion Masaveu SA',
 'SP_PT_CRH PLC',
 'SP_PT_HeidelbergCement AG',
 'SP_PT_InterCement Participacoes SA',
 'SP_PT_LafargeHolcim Ltd',
 'SP_PT_Secil Companhia Geral de Cal e Cimento SA',
 'SP_PT_start',
 'SP_PT_Unknown',
 'SP_PT_Votorantim SA',
 'UK_Breedon Group PLC',
 'UK_CEMEX SAB de CV',
 'UK_CRH PLC',
 'UK_HeidelbergCement AG',
 'UK_LafargeHolcim Ltd',
 'UK_Mannok Build Ltd',
 'UK_start - Kopie']

In [3]:
number = 3
#Casenames = ["UK_cement"]
CaseName = Casenames[number-1]

print (CaseName)
print (number)
#pathCASE = prepare_CASE.make_NewCase(CaseName)
#pathCASE = f'../Example_Case/{CaseName}/'
pathCASE = f'../No_collaboration/{CaseName}/'
print(pathCASE)

SP_PT_CEMEX SAB de CV
3
../No_collaboration/SP_PT_CEMEX SAB de CV/


In [4]:
change_delimiter_for_all_csv_in_path(f'{pathCASE}input/',separator=',')

### Make CSV files readable for German Excel so they can be altered. 

Create gridded distances between points:

In [5]:
prepare_CASE.create_GriddedDistanceCSV(pathCASE, CaseName)

### STEP 2 of 2: DEFINE optimisation runs

In [6]:
RunName = 'Technology'
d = {
    # info on instantiating the RTN model
    "instantiate": {
        # True/False: install CC for existing sites (see csv file)
        "install_CC": 0,
        # True/False: only solve for CO2 transport and storage, + NETs
        "CCS_network_only": 0,
        # ["TLC"]: str, linked to objective function
        "obj": "TLC"
    },
    # info on optimisation runs, i.e. state gates
    "stateGates": {
        # True/False: apply backcasting (rather than snapshot) for subsequent runs?
        "backcasting" : 0,
        # array: net CO2 reduction targets in % for each state gate (run)
        "targets": [100]
    },
    # info on solver
    "solver": {
        # ["glpk", "cplex"]: solver name
        "name": "cplex",
        # float: termination criterion for mipgap, i.e. optimality. default: 0.05
        "mipgap": 0.0002,
        # int: timeout limit in seconds
        "tlimit": 14400 ,#7200,#28000(7.7h)
        # Only works for CPLEX - tolerances between 0.0 and 0.5 default 1E-9:
        "integralityTolerance": 0 #1E-50
    }
}

In [7]:
prepare_CASE.write_RunInfo(pathCASE, RunName, d)

Create input CSVs

In [8]:
# load data
df = prepare_CASE.load_ComponentModels(f'{pathCASE}input/')
# add weight
df['OBJ_WEIGHT'] = 1
# specify output
fs = {'NAME_PLATE_CAP': [],
      'MIN_CAP': [],
      'INV_COEFF': ['OBJ_WEIGHT'],
      'PROCESS_COEFF': ['OBJ_WEIGHT'], 
      'RESOURCE_CONV_RATE': []}
# extractcsv files
pathWrite = f'{pathCASE}input/'
for param, ks in fs.items():
    prepare_CASE.write_csvData(pathWrite, df, param, ks)

Look up transport costs

In [9]:
filePath_transport = "../../INDiECAR-BT/Results/Transport_calculations/"
prepare_CASE.prepare_transport_data(f'{pathCASE}input/', filePath_transport)

     OID    OLIVINE    BIOMASS       CLAY  Grid_id
0  CID82  11.035338  21.062831   8.987746        1
1  CID83  16.573763  28.782357  14.173735        2
2  CID84   8.998351  19.025845   6.950759        3
3  CID85  13.328018  23.355511  11.280426        4
4  CID86  24.818694  34.846188  22.770766        5


Lastly, we change all delimiters to ';' to be opened in Excel (Germany). 

In [10]:
change_delimiter_for_all_csv_in_path(f'{pathCASE}input/',separator=';')